#GNR 401 - Remote sensing and Image processing

**Problem objective**: Implementation of fundamental image processing and enhancement
algorithms to a hyperspectral image cube.

In [1]:
# Import basic libraries used for calculations
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from IPython.display import display # For interactive operations
import ipywidgets as widgets

In [2]:
# Load the uploaded .mat file
data = loadmat('Indian_pines.mat')
cube = None
for k,v in data.items():
    if isinstance(v, np.ndarray) and v.ndim == 3:
        cube = v
        cube_key = k
        break
if cube is None:
    raise RuntimeError('No 3D array found in MAT file. Check uploaded file contents.')
H,W,B = cube.shape
print('Found hyperspectral cube variable:', cube_key, 'shape=', cube.shape)

Found hyperspectral cube variable: indian_pines shape= (145, 145, 220)


In [3]:
# Normalizing the image data to floating-point values between 0 and 1
def normalize_float(img):
    arr = np.asarray(img, dtype=np.float64)
    mn = arr.min()
    mx = arr.max()
    if mx == mn:
        return np.zeros_like(arr, dtype=np.float64)
    return (arr - mn) / (mx - mn)

# Converts a normalized float image (0–1) into 8-bit image (0–255)
def to_uint8(img_float):
    return np.clip((img_float * 255.0), 0, 255).astype(np.uint8)

# Displays a normalized grayscale image (values in 0–1) using Matplotlib
def imshow_gray_float(img, title=None, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.imshow(img, cmap='gray', vmin=0, vmax=1)
    if title: plt.title(title)
    plt.axis('off')
    plt.show()

##Log Transformation


$s = c \cdot \log(1+r)$

  c = constant

In [4]:
def log_transform(img, c=None):
    r = normalize_float(img)

    s = np.log(1 + r)

    # auto-scale if c not given
    if c is None:
        c = 1.0 / (s.max() + 1e-12)

    s = c * s
    return np.clip(s, 0, 1)

##Power Law(Gamma) Transformation

$s = c/r^{\gamma}$

c and γ are both positive constants

In [5]:
def gamma_correction(img, gamma=1.0, c=None):

    r = normalize_float(img)

    s = np.power(r, gamma)

    # auto-scale if c is not given
    if c is None:
        c = 1.0 / (s.max() + 1e-12)
    s = c * s

    return np.clip(s, 0, 1)

##Contrast Stretching (Min–Max)

(r1,s1) = (rmin , 0) and (r2,s2) = (rmax  , L-1)

In [6]:
def contrast_stretch(img, p_low=2, p_high=98):

    arr = np.asarray(img, dtype=np.float64)

    rmin = np.percentile(arr, p_low)
    rmax = np.percentile(arr, p_high)

    # Avoid division by zero
    if np.isclose(rmax, rmin):
        return np.zeros_like(arr)

    # Stretch pixel values
    s = (arr - rmin) / (rmax - rmin)

    return np.clip(s, 0, 1)

## Histogram Equalization

Let:
*   $r_k =$ the $k$-th intensity level ($0 \le k \le 255$)
*   $n_k =$ number of pixels with intensity $r_k$
*   $L =$ total possible intensity levels (e.g., $256$)
*   $n =$ total number of pixels

1.  **Compute the Probability Density Function (PDF):**
    $$p(r_k) = \frac{n_k}{n}$$

2.  **Compute the Cumulative Distribution Function (CDF):**
    $$c(r_k) = \sum_{j=0}^{k} p(r_j)$$

3.  **Apply the Transformation:**
    $$s_k = (L-1) \times c(r_k)$$
    $s_k$ is the new intensity corresponding to $r_k$.

4.  **Map each original pixel intensity** $r_k \to s_k$.


In [7]:
def histogram_equalization(img, nbins=256):
    r = normalize_float(img)

    # Scale pixels to [0, nbins-1] and flatten to 1D
    # flat will contain integer bin indices for each pixel
    flat = (r * (nbins - 1)).astype(np.int32).ravel()
    hist = np.bincount(flat, minlength=nbins).astype(np.float64)

    cdf = hist.cumsum()
    cdf_min = cdf.min()
    cdf_max = cdf.max()

    if cdf_max == cdf_min:
        return np.zeros_like(r)
    cdf_norm = (cdf - cdf_min) / (cdf_max - cdf_min)

    out_flat = cdf_norm[flat]
    out = out_flat.reshape(r.shape)
    return np.clip(out, 0, 1)

In [8]:
# It adds py pixels of padding on the top and bottom
# It adds px pixels of padding on the left and right
# The padding uses reflection (mirror effect)
def pad_reflect(arr, py, px):
    return np.pad(arr, ((py,py),(px,px)), mode='reflect')

In [9]:
def convolve2d(image, kernel):

    img = np.asarray(image, dtype=np.float64)
    k = np.asarray(kernel, dtype=np.float64)
    ky, kx = k.shape
    py, px = ky//2, kx//2
    padded = pad_reflect(img, py, px)

    k_flip = np.flip(np.flip(k, axis=0), axis=1)

    H, W = img.shape
    out = np.zeros_like(img, dtype=np.float64)

    for y in range(H):
        for x in range(W):
            patch = padded[y:y+ky, x:x+kx]
            out[y,x] = np.sum(patch * k_flip)
    return out

def mean_filter_kernel(size):
    return np.ones((size,size), dtype=np.float64) / (size*size)

def sobel_kernels():
    kx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]], dtype=np.float64)
    ky = np.array([[1,2,1],[0,0,0],[-1,-2,-1]], dtype=np.float64)
    return kx, ky

def prewitt_kernels():
    kx = np.array([[-1,0,1],[-1,0,1],[-1,0,1]], dtype=np.float64)
    ky = np.array([[1,1,1],[0,0,0],[-1,-1,-1]], dtype=np.float64)
    return kx, ky


This code builds an interactive Jupyter dashboard using sliders to apply and compare different image enhancement techniques on hyperspectral image bands for better visualization

In [10]:
# Interactive dashboard
band_slider = widgets.IntSlider(value=B//2, min=0, max=B-1, step=1, description='Band')
gamma_slider = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description='Gamma')
log_c_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Log c')
mean_kernel_slider = widgets.IntSlider(value=5, min=1, max=15, step=2, description='Mean k')

out = widgets.Output()

def update_display(change=None):
    with out:
        out.clear_output()
        b = band_slider.value
        img = cube[..., b].astype(np.float64)
        # original
        orig = normalize_float(img)
        # transforms
        log_img = log_transform(img, c=log_c_slider.value)
        gamma_img = gamma_correction(img, gamma=gamma_slider.value)
        cs = contrast_stretch(img)
        he = histogram_equalization(img)
        # convolution mean
        ksize = mean_kernel_slider.value
        if ksize % 2 == 0:
            ksize += 1
        k_mean = mean_filter_kernel(ksize)
        mean_out = convolve2d(normalize_float(img), k_mean)
        mean_out = normalize_float(mean_out)
        # gradient (sobel)
        kx, ky = sobel_kernels()
        gx = convolve2d(normalize_float(img), kx)
        gy = convolve2d(normalize_float(img), ky)
        grad = np.sqrt(gx**2 + gy**2)
        grad = normalize_float(grad)
        # plot a 2x4 grid
        fig, axs = plt.subplots(2,4, figsize=(16,8))
        axs[0,0].imshow(orig, cmap='gray', vmin=0, vmax=1); axs[0,0].set_title(f'Original b={b}'); axs[0,0].axis('off')
        axs[0,1].imshow(log_img, cmap='gray', vmin=0, vmax=1); axs[0,1].set_title('Log'); axs[0,1].axis('off')
        axs[0,2].imshow(gamma_img, cmap='gray', vmin=0, vmax=1); axs[0,2].set_title(f'Gamma {gamma_slider.value}'); axs[0,2].axis('off')
        axs[0,3].imshow(cs, cmap='gray', vmin=0, vmax=1); axs[0,3].set_title('Contrast Stretch'); axs[0,3].axis('off')
        axs[1,0].imshow(he, cmap='gray', vmin=0, vmax=1); axs[1,0].set_title('Hist Equalized'); axs[1,0].axis('off')
        axs[1,1].imshow(mean_out, cmap='gray', vmin=0, vmax=1); axs[1,1].set_title(f'Mean {ksize}x{ksize}'); axs[1,1].axis('off')
        axs[1,2].imshow(grad, cmap='gray', vmin=0, vmax=1); axs[1,2].set_title('Sobel Grad Mag'); axs[1,2].axis('off')
        # pseudo-RGB from three chosen bands (static choices)
        r_idx = min(B-1, 60)
        g_idx = min(B-1, 30)
        b_idx = min(B-1, 10)
        rgb = np.zeros((H,W,3), dtype=np.float64)
        rgb[...,0] = normalize_float(cube[..., r_idx])
        rgb[...,1] = normalize_float(cube[..., g_idx])
        rgb[...,2] = normalize_float(cube[..., b_idx])
        axs[1,3].imshow(to_uint8(rgb))
        axs[1,3].set_title(f'Pseudo-RGB {r_idx},{g_idx},{b_idx}'); axs[1,3].axis('off')
        plt.tight_layout(); plt.show()

# connect events
band_slider.observe(update_display, names='value')
gamma_slider.observe(update_display, names='value')
log_c_slider.observe(update_display, names='value')
mean_kernel_slider.observe(update_display, names='value')

# display controls and initial output
controls = widgets.HBox([band_slider, gamma_slider, log_c_slider, mean_kernel_slider])
display(controls)
display(out)
update_display()

Output()